In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
%reload_ext autoreload

In [3]:
import logging
import csv
import sys
from pathlib import Path
import subprocess


In [4]:
logger = logging.getLogger(__name__)
logger.root.setLevel('DEBUG')

In [5]:
def csv_to_list(file):
    '''read csv file `file` into a list
    
    Guess the CSV dialect (e.g. tsv, csv, etc.)
    
    Returns `list`'''
    csvFile = Path(file).expanduser().absolute()
    file_csv = []
    
    with open(csvFile, 'r') as file:
        dialect = csv.Sniffer().sniff(file.read(1024))
        file.seek(0)
        reader = csv.reader(file, dialect)
        for row in reader:
            file_csv.append(row)

    return file_csv

In [6]:
def map_headers(csv_list, expected_headers=[]):
    '''map row 0 of a csv as formatted as a list to a dictionary of expected header values'''
    missing_headers = []
    header_map = {}
    
    csvHeader = csv_list[0]
    
    logger.debug('checking for missing headers')
    for each in expected_headers:
        if each not in csvHeader:
            missing_headers.append(each)
            
    if len(missing_headers) > 0:
        logging.warning(f'missing expected headers: {missing_headers}')
    for index, value in enumerate(csvHeader):
        if value in expected_headers:
            header_map[value] = index
        
    logging.debug('completed mapping')
    return(header_map, missing_headers)

In [7]:
def do_exit(e='unknown error in unknown module: BSoD!', exit_status=0, testing=False):
    logging.info(f'exited before completion with exit code {exit_status}')
    print('program exited due to errors')
    print(e)
    if not testing:
        sys.exit(exit_status)

In [191]:
class student_path(gd_path):
    def __init__(self, class_of=None, id_number=None, name=None):
        super(student_path, self).__init__()
#         self.student_dir_name = None
        self.path_parts = {'ClassOf': None, 'id_number': None, 'name': None}
        self.class_of = class_of
        self.name = name
        self.id_number = id_number
    
    def get_xattr(self, attribute, file=None):
        if not file:
            file = self.student_dir_name
        return super().get_xattr(attribute, file)
    
    @property
    def class_of(self):
        return self._class_of
    
    @class_of.setter
    def class_of(self, class_of):
        if not class_of:
            self._class_of = None
        else:    
            if not isinstance(class_of, int):
                raise TypeError('class_of must be of type `int`')
        self.path_parts['ClassOf'] = f'ClassOf-{class_of}'
        self._class_of = class_of
        
    @property
    def name(self):
        return self._name
    
    @name.setter
    def name(self, name):
        if not name:
            self._name = None
        else:
            if not isinstance (name, str):
                raise TypeError('name must be of type `str`')
        self.path_parts['name'] = name
        self._name = name
        
    @property
    def id_number(self):
        return self._id_number
    
    @id_number.setter
    def id_number(self, number):
        if not number:
            self._id_number = None
        else:
            if not isinstance (number, int):
                raise TypeError('id_number must be of type `int`')
        self.path_parts['id_number'] = number
        self._id_number = number

    @property
    def student_dir_name(self):
        d = f"/{self.path_parts['ClassOf']}/{self.path_parts['name']} - {self.path_parts['id_number']}"
        if self.root:
            # not sure why this is needed, but any joining of self.root/Path(d) fails
            d = f'{str(self.root)}/{d}'
        return Path(d)
    
    # method for checking for similarly named student folders in this ClassOf folder
    def check_similar(self):
        '''check for similarly named directories based on student id number
        
        Returns:
            `dict` of name, web_view_link''' 
        matches = {}
        for i in self.student_dir_name.parent.glob(f"*{self.path_parts['id_number']}*"):
            match_id = self.get_xattr('user.drive.id', self.student_dir_name.parent/i)
            if i.absolute().is_dir():
                url = '/'.join((self._dir_base, match_id[0]))
            else:
                url = '/'.join((self._file_base, match_id[0]))
            matches[i] = url
        return matches
#         return i
            
        

In [192]:
s = student_path(id_number=12345)
s.path = '/Volumes/GoogleDrive/Shared drives/IT Blabla I/Student Cumulative Folders (AKA Student Portfolios)'
s.class_of = 1000
s.name = "a, b"
s.id_number = 123456

In [194]:
s.path

PosixPath('/Volumes/GoogleDrive/Shared drives/IT Blabla I/Student Cumulative Folders (AKA Student Portfolios)')

In [196]:
print(s.webview_link)

https://drive.google.com/drive/folders/1QVmW_J0paYdWKN5MONTzfgZWAsUiGfu7


In [155]:
i = s.check_similar()

In [156]:
i

{PosixPath('/Volumes/GoogleDrive/Shared drives/IT Blabla I/Student Cumulative Folders (AKA Student Portfolios)/ClassOf-1000/a, b - 123456'): 'https://drive.google.com/drive/folders//1QVmW_J0paYdWKN5MONTzfgZWAsUiGfu7',
 PosixPath('/Volumes/GoogleDrive/Shared drives/IT Blabla I/Student Cumulative Folders (AKA Student Portfolios)/ClassOf-1000/a-c-d III, b - 123456'): 'https://drive.google.com/drive/folders//1QYDXG8H941_04mmEV_IhK5WiQMsTmgtv'}

In [98]:
if len(s.check_similar()) > 1:
    print(f'WTF? Why are there more than one folder for student: {s.name}')

WTF? Why are there more than one folder for student: a, b


In [95]:
s.student_dir_name

PosixPath('/Volumes/GoogleDrive/Shared drives/IT Blabla I/Student Cumulative Folders (AKA Student Portfolios)/ClassOf-1000/a, b - 123456')

In [77]:
s.get_xattr('user.drive.id')

['1j83z1VVKtpKdy4zO3m75laUazZgXhMK6']

In [281]:
class gd_path():
    def __init__(self, path=None):
        '''google drive path class
        
        Attributes:
            path(`str`): path to google drive drive object'''
        self.confirmed = False
        self.path = path
        self._file_base = 'https://drive.google.com/file/d/'
        self._dir_base = 'https://drive.google.com/drive/folders/'
        self.is_file = False   
    
    @property
    def path(self):
        return self._path
    
    @path.setter
    def path(self, path):
        '''full path to object
        
        Args:
            path(`str` or `Path`): /path/to/object
            
        Sets Attributes:
            self.path: path to object
            self.root: same as path for directories, parent directory for files
            self.is_file: true for files and file-like objects, false for directories'''
        if not path:
            self._path = None
        else:
            self._path = Path(path)
            if self._path.is_dir() and self._path.exists():
                self.root = self._path
                self.is_file = False
            if self.path.is_file() and self._path.exists():
                self.root = self._path.parent
                self.is_file = True
            
            if not self._path.exists():
                self.is_file = False
                self.root = self._path.parent

    @property
    def webview_link(self):
        '''full webview link to object in google drive'''
        self._webview_link = None
        try:
            item_id = self.get_xattr('user.drive.id')
        except FileNotFoundError as e:
            logging.debug(f'{e}')
            return None
        except ChildProcessError as e:
            logging.debug(f'{e}')
            return None

        if len(item_id) < 1:
            return None
        else:
            item_id = item_id[0]
        
        
        if not self.is_file:
            self._webview_link = f'{self._dir_base}{item_id}'
        if self.is_file:
            self._webview_link = f'{self._file_base}{item_id}'
        return self._webview_link
            
    def check_parent(self, expected):
        '''checks if the parent matches the expected parent'''
        if self.root.parents[0].name == expected:
            return True
        else:
            return False
        
    def get_xattr(self, attribute, file=None):
        '''get the extended attributes of a file or directory
        Args:
            file(`str` or Path): path to file
            attribute('`str`'): attribute key to access

        Returns:
            `list` - attribute or key: attribute pairs

        Raises:
            FileNotFoundError - file or directory does not exist
            ChildProcessError - xattr utility exits with non-zero code 
                This is common for files that have no extended attributes or do not
                have the requested attribute'''
        if not file:
            file = self.path
        else:
            file = Path(file).absolute()
            
        attributes = []
        if not file.exists():
            raise FileNotFoundError(file)

        p = subprocess.Popen(f'xattr -p  {attribute} "{file.resolve()}"', shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        for line in p.stdout.readlines():
            attributes.append(line.decode("utf-8").strip())
    #         attributes = attributes + line.decode("utf-8").strip()
        retval = p.wait()
        if retval != 0:
            raise ChildProcessError(f'xattr exited with value: {retval}')
        return attributes     

    @property
    def file_id(self, path=None):
        '''unique file id for each object (directories or file)
        
        Args:
            path(`str` or `Path`): path to object; defaults to self.path
        
        Returns:
            `list` of `str` containing the file id'''
        if not path:
            path = self.path
        try:
            file_id = self.get_xattr('user.drive.id', path)
        except FileNotFoundError as e:
            logging.info(f'{path} does not appear to exist; cannot get attributes')
            file_id = None
        return file_id
    
    def confirm(self, path=None):
        '''confirm that a created object has been sent over file stream
        
        Args:
            path(`str` or `Path`): path to object; default is self.path
        
        Returns:
            `list` of `str` containing the file id
            
        Attributes Set:
            self.confirmed: True when object has been sent'''
        
        if not path:
            path = self.path
        file_id = self.file_id
        
        if file_id:
            if 'local-' in file_id[0]:
                self.confirmed = False
                file_id = None
            else:
                self.confirmed = True
        return file_id
    
    def mkdir(self, path=None, parents=False, exist_ok=False, kwargs={}):
        '''create a directory using pathlib.Path().mkdir()
        
        Args:
            path(`str` or `Path`): path to create
            parents(`bool`): create parent directories - default false
            exists_ok(`bool`): do not raise error if directory exists
            kwargs: kwargs for pathlib.Path().mkdir()
            
        Returns:
            file_id(`list`)'''
        if not path:
            path = self.path
            
        if path.is_file():
            raise TypeError(f'{path} is a file')
            
        path = Path(path)
            
        self.path.mkdir(parents=parents, exist_ok=exist_ok, **kwargs)
        if self.confirm(path):
            file_id = self.get_xattr('user.drive.id', self.path)
        return self.file_id
    

In [283]:
g = gd_path(path='/Volumes/GoogleDrive/Shared drives/IT Blabla I/xyz-pdq/spam/eggs_ham_spam')
m = gd_path(path='/Volumes/GoogleDrive/My Drive/dev.csv')

In [285]:
g.file_id

INFO:root:/Volumes/GoogleDrive/Shared drives/IT Blabla I/xyz-pdq/spam/eggs_ham_spam does not appear to exist; cannot get attributes


In [286]:
g.confirm()

INFO:root:/Volumes/GoogleDrive/Shared drives/IT Blabla I/xyz-pdq/spam/eggs_ham_spam does not appear to exist; cannot get attributes


In [287]:
g.mkdir(exist_ok=True)

['local-223141']

In [291]:
g.confirm()

['1Qpzp14230uUtd9GD4qjuVadxLK_Z4nXz']

In [210]:
print(g.webview_link)
print(m.webview_link)

DEBUG:root:/Volumes/GoogleDrive/Shared drives/IT Blabla I/xyz-pdq


None
https://drive.google.com/drive/folders/1vF1_x-_4dKRaPme9o0ZGuJO2pCMdTell


In [212]:
g.path

PosixPath('/Volumes/GoogleDrive/Shared drives/IT Blabla I/xyz-pdq')

In [52]:
m.root.is_dir()

True

In [369]:
g.path.mkdir()

FileExistsError: [Errno 17] File exists: '/Volumes/GoogleDrive/Shared drives/IT Blabla I/xyz-pdq/spam/eggs_ham_spam'

In [367]:
def validate_data(csv_list, expected_headers, header_map):
    valid = []
    invalid = []

    for row in csv_list[1:]:
        good_row = True
        for k in expected_headers.keys():
            # test for coercable types
            try:
                test = expected_headers[k](row[header_map[k]])
            except ValueError:
#                 do_exit(f'Bad student.export: {k} contained {row[header_map[k]]}\ncannot continue. Please try running the export again.')
                logging.warning(f'{row}')
                logging.warning(f'Bad student.export: column "{k}" contained "{row[header_map[k]]}"--this should be {(expected_headers[k])}')
                invalid.append(row)
                good_row = False
                break
        if  good_row:
            valid.append(row)
        
    return valid, invalid
            
    

In [368]:
v, i = validate_data(c, e, m)

In [ ]:
print(g.get_xattr('user.drive.team_drive_id'))
print(m.get_xattr('user.drive.id'))
# print(m.get_xattr('user.drive.team_drive_id'))

In [293]:
# def create_student_folders(csv_file, drive_path):

In [374]:
def main():
    
#     csv_file = Path('./student.export.csv.text')
    csv_file = Path('./bad.export.csv.text')
    
    drive_path = Path('/Volumes/GoogleDrive/Shared drives/IT Blabla I/')
    
#     expected_headers = ['LastFirst', 'ClassOf', 'Student_Number']
    expected_headers ={'LastFirst': str, 'ClassOf': int, 'Student_Number': int}
    
    ############
    
    logging.debug('starting up')
    
    # check drive path is on a shared drive 
    google_drive = gd_path(drive_path)
    try:
        google_drive.get_xattr('user.drive.id')
    except ChildProcessError as e:
        do_exit(f'The specified Google Drive "{drive_path}" is not a Shared Drive', 1)
    except FileNotFoundError as e:
        do_exit(f'The specified Google Shared Drive {drive_path}" does not exist', 1)
    
    # check if export is OK
    try:
        csv_list = csv_to_list(csv_file)
    except (FileNotFoundError, OSError, IOError) as e:
        logging.error(f'could not read csv file: {csv_file}')
        logging.error(f'{e}')
        do_exit(e, 1)
    
    # check for headers in CSV file
    header_map, missing_headers = map_headers(csv_list, expected_headers.keys())
    
    if len(missing_headers) > 0:
        do_exit(f'{csv_file.name} is missing one or more headers:\n\t{missing_headers}', 1)
    
    # validate the csv list
    valid_rows, invalid_rows = validate_data(csv_list, expected_headers, header_map)
    


In [375]:
f = main()

DEBUG:root:starting up
DEBUG:__main__:checking for missing headers
DEBUG:root:completed mapping


In [ ]:
print( f)

In [ ]:
int(f[1][10])